In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None

In [2]:
import csv
with open('../drugs.csv', 'r') as g:
    reader = csv.reader(g)
    drugs = list(reader)[0]

print(drugs)

['Fentanyl', 'Norfentanyl', 'Oxycodone', 'Noroxycodone', 'Hydrocodone', 'Norhydrocodone', 'Codeine', 'Morphine', 'Dihydrocodeine']


In [4]:
f = '../acutis_current.csv'
df=pd.read_csv(f,low_memory=False)
df['COLLECT_DATE']=pd.to_datetime(df['COLLECT_DATE'],infer_datetime_format=True,errors='coerce')


In [5]:
#I only want patients who had their first appointment in April or later
df=df[df['COLLECT_DATE'] >= pd.to_datetime('04/01/2019',infer_datetime_format=True)].copy()
print("Shape:",df.shape)
print("Columns:",df.columns.to_list())

Shape: (28925, 28)
Columns: ['PATIENT_ID', 'PATIENT_L_NAME', 'PATIENT_F_NAME', 'PATIENT_M_NAME', 'PATIENT_GENDER', 'PATIENT_BIRTH', 'PRESCRIBED_MEDICATIONS', 'ACCESSION', 'DOCTOR_ID', 'DOCTOR_L_NAME', 'DOCTOR_F_NAME', 'DOCTOR_M_NAME', 'REQ_DATE', 'COLLECT_DATE', 'RECEIVED_DATE', 'TEST_NAME', 'TEST_UNITS', 'RESULT_FLAG', 'RESULT_NUMERIC', 'RESULT_CREATED_DATE', 'SPECIMEN TYPE', 'date', 'latest', 'PATIENT_FLAG', 'SCREEN_FLAG', 'CHIP', 'bup', 'bad']


In [6]:
#Combine the Patient name columns into 
def createname(row):
    fname = row['PATIENT_F_NAME']
    lname = row['PATIENT_L_NAME']
    name = str(lname)+', '+str(fname)
    return name

df['PATIENT_NAME']=df.apply(createname,axis=1)

In [7]:
df1=df.copy()
df2=df.copy()
df2.set_index(['PATIENT_ID'],inplace=True)
df2.sort_index(inplace=True)

In [8]:
df1=df1[['PATIENT_ID','PATIENT_NAME','COLLECT_DATE','TEST_NAME','RESULT_FLAG','RESULT_NUMERIC']]
df2=df2[['COLLECT_DATE']]

In [11]:
#We want to get a flag if the sample is the first one on record
#After some testing, this yeilds slightly faster results than writing a function and using df.apply()
#A version of this cell written as a function is located in the cell below
import time
start = time.time()

lff=[]
for it in df1.itertuples():
    ID=it.PATIENT_ID
    try:
        earl=np.min(df2.loc[ID,'COLLECT_DATE'])
    except TypeError:
        earl=np.min(np.array(df2.loc[ID,'COLLECT_DATE']))
#     letd.append(earl)
    if earl==it.COLLECT_DATE:
        lff.append(1)
    else:
        lff.append(0)
arrff=np.array(lff)
df1['FIRST_FLAG']=arrff
del(earl,lff,arrff)
print("-"*6,time.time() - start,"-"*6)

------ 33.27493214607239 ------


In [13]:
#We want rows who are on the date of the first urine sample for the patient, are testing for substances on the list of drugs, 
# and are on or after the first of June (so that way we know its actually the first sample and there likely wasnt one before May 1)
dffirst=df1[(df1['FIRST_FLAG']==1) & (df1['TEST_NAME'].isin(drugs)) & (df1['COLLECT_DATE'] >= pd.to_datetime('June 1, 2018',infer_datetime_format=True))].copy()
dffirst.set_index(['PATIENT_ID','RESULT_FLAG'],inplace=True)
dffirst.sort_index(inplace=True)

In [14]:
#sort names into lists, save these lists to csvs
d=[]
nd=[]
for i in range(dffirst.shape[0]):
    ID=dffirst.index[i][0]
    try:
        dffirst.loc[ID,'Positive']
        d.append(ID)
    except KeyError:
        nd.append(ID)
da=np.array(d)
nda=np.array(nd)
da=np.unique(da)
nda=np.unique(nda)
allids=np.unique(df['PATIENT_ID'])
daandnda=np.concatenate((nda,da))
others=np.setdiff1d(allids,daandnda)

In [16]:
allids.shape[0]

974

In [21]:
posdf=pd.DataFrame({'Patient':da,'Status':np.array(['Positive']*da.shape[0])})
negdf=pd.DataFrame({'Patient':nda,'Status':np.array(['Negative']*nda.shape[0])})
result=pd.concat([posdf,negdf])
result.sort_values('Patient',inplace=True)
result.reset_index(drop=True,inplace=True)

In [30]:
result.head()

,Patient,Status
0,1840-46471846,Positive
1,1840-46679030,Negative
2,1840-46754094,Positive
3,1840-46834577,Negative
4,1840-46946018,Negative


Note that the first patient listed, 1840-46471846, tested positive for Norfentanyl on their first visit, as seen below:

In [38]:
df1[(df1['PATIENT_ID'].isin(['1840-46471846']))&(df1['FIRST_FLAG']==1)][['PATIENT_ID','TEST_NAME','RESULT_FLAG']]

,PATIENT_ID,TEST_NAME,RESULT_FLAG
138877,1840-46471846,Gabapentin,Negative
138878,1840-46471846,Pregabalin,Negative
138879,1840-46471846,Fentanyl,Negative
138880,1840-46471846,Norfentanyl,Positive
138881,1840-46471846,Heroin Metabolite - 6-AM,Positive
138882,1840-46471846,Buprenorphine,Negative
138883,1840-46471846,Codeine,Positive
138884,1840-46471846,Dihydrocodeine,Negative
138885,1840-46471846,Hydrocodone,Negative
138886,1840-46471846,Hydromorphone,Positive


In [48]:
fo='PosAndNegUrine.csv'
result.to_csv(fo,index=False)